In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.realpath('..'))

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import plotly.express as ex
import plotly.graph_objects as go
import IPython.display as ipython

from tracking_v2.target import NearConstantVelocityTarget
from tracking_v2.tracker.mht import MultiTargetTracker, MhtDecisionClassifier, MhtEstimationMetrics
from tracking_v2.sensor import GeometricSensor
from tracking_v2.evaluation.runner import TrackerRunner
from tracking_v2.evaluation.plot.mht import MhtInspectCallback, plot_mht
from tracking_v2.evaluation.plot.filter import NScoreEvaluationResult, plot_nscore

from tracking_v2.util import to_df, display

In [3]:
display.as_png = False

## Parallel targets

In [4]:
target_1 = NearConstantVelocityTarget(velocity=[0, 20, 0], noise_intensity=1)
target_2 = NearConstantVelocityTarget(initial_position=[10, 0, 0], velocity=[0, 20, 0], noise_intensity=1)

data, nees = [], []

for R_sigma in [.5, 1, 1.2, 1.5, 2, 5, 10, 15, 20, 25, 30]:
#for R_sigma in [.5, 10]:
    print(f"R_sigma = {R_sigma}")
    sensor = GeometricSensor(R=np.eye(3) * R_sigma**2)
    
    mtt = MultiTargetTracker()
    icb = MhtInspectCallback()
    dcb = MhtDecisionClassifier()
    mcb = MhtEstimationMetrics()
    runner = TrackerRunner([target_1, target_2], [sensor], mtt, [icb, dcb, mcb])
    
    runner.run_many(100, 200)
    data.append((R_sigma, dcb.inconsistent_init_many[:, 1].mean(), np.std(dcb.inconsistent_init_many[:, 1]), np.max(dcb.inconsistent_init_many[:, 1]),
                          dcb.already_taken_many[:, 1].mean(), np.std(dcb.already_taken_many[:, 1]), np.max(dcb.already_taken_many[:, 1]),
                          dcb.better_score_many[:, 1].mean(), np.std(dcb.better_score_many[:, 1]), np.max(dcb.better_score_many[:, 1])))
    nees.append((R_sigma, mcb.nees))
    #print(mcb.track_to_target)
    #print(dcb)

data = np.asarray(data)

R_sigma = 0.5
R_sigma = 1
R_sigma = 1.2
R_sigma = 1.5
R_sigma = 2
R_sigma = 5
R_sigma = 10
R_sigma = 15
R_sigma = 20
R_sigma = 25
R_sigma = 30


In [5]:
to_df(data, columns=['R_sigma', 'init', 'init.std', 'init.max', 'at', 'at.std', 'at.max', 'bs', 'bs.std', 'bs.max'])

,R_sigma,init,init.std,init.max,at,at.std,at.max,bs,bs.std,bs.max
0,0.5,0.08,0.391918,2.0,0.07,0.353695,3.0,0.07,0.353695,3.0
1,1.0,0.18,0.572364,2.0,0.28,1.166876,10.0,0.28,1.166876,10.0
2,1.2,0.20,0.600000,2.0,0.34,1.250760,10.0,0.34,1.250760,10.0
3,1.5,0.26,0.672607,2.0,0.45,1.344433,10.0,0.45,1.344433,10.0
4,2.0,0.34,0.751266,2.0,0.60,1.363818,10.0,0.60,1.363818,10.0
5,5.0,0.58,0.907524,2.0,2.03,2.051609,9.0,2.03,2.051609,9.0
6,10.0,0.90,0.994987,2.0,3.32,2.340427,9.0,3.32,2.340427,9.0
7,15.0,0.97,0.994535,2.0,4.98,3.982411,24.0,4.97,3.973550,24.0
8,20.0,1.00,0.979796,2.0,5.99,4.198797,25.0,5.99,4.189260,25.0
9,25.0,1.00,0.948683,2.0,7.47,4.844492,23.0,7.66,4.771205,23.0
